#how to setup rtmp server, please reference below url
https://www.cnblogs.com/nowgood/p/ffmpegnginx.html

#send rtmp data from pi4 to server, can use follow command
ffmpeg -f v4l2 -r 24 -video_size vga -pix_fmt yuv420p12be -i /dev/video0 -b:v 500k -c:v h264_omx -preset ultrafast -an -f flv rtmp://52.80.130.52/live/123456


In [ ]:
!pip install gluoncv -i https://opentuna.cn/pypi/web/simple

In [1]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
output_notebook()

Loading BokehJS ...

In [2]:
# import sagemaker
# sagemaker_endpoint_name = 'object-detection-2019-10-18-17-56-41-925'
# sagemaker_endpoint = sagemaker.predictor.RealTimePredictor(sagemaker_endpoint_name)
# sagemaker_endpoint.content_type = 'image/jpeg'
from object_detection import ObjectDetection
objectDetection = ObjectDetection()

for i in range(10):
    # use GluonCV
    detect_start = time.time()
    frame = cv2.imread('../images/shenzhen_v1/frame_1604547852.3077297.jpg')
    class_IDs, scores, bounding_boxes = objectDetection.detect_image(frame)
#             class_IDs, scores, bounding_boxes = objectDetection.detect_image_yolo(frame)
    #print('class_IDs:', class_IDs)
    #print('scores:', scores)
    #print('bounding_boxes:', bounding_boxes)
    detect_end = time.time()
    print('detect time:', detect_end-detect_start)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/gluon/block.py:1454: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


detect time: 38.04889917373657
detect time: 0.07523345947265625
detect time: 0.08372020721435547
detect time: 0.07837033271789551
detect time: 0.08886027336120605
detect time: 0.0778815746307373
detect time: 0.07823562622070312
detect time: 0.09227919578552246
detect time: 0.08020663261413574
detect time: 0.08249139785766602


In [3]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
#         frame=cv2.flip(frame, -1) # because Bokeh flips vertically
        frame=cv2.flip(frame, 0) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
#         frame=cv2.flip(frame, -1)
        frame=cv2.flip(frame, 0) 
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [4]:
def detection_result_process(frame, classes, class_IDs, scores, bounding_boxes, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, max_item_cnt, pre_msg, pre_msg2):
    thres = 0.45
    max_thres = 0.7
    if class_IDs is not None and len(class_IDs) == 1:
            if len(class_IDs[0]) >= 1:
                hand = False
                item_cnt = [0, 0]
                for i in range(len(class_IDs[0])):
                    if scores[0][i] > thres:
                        class_ID = int(class_IDs[0][i])
                        score = float(scores[0][i])
                        bounding_box = bounding_boxes[0][i]
                        # print('class_ID:', class_ID, 'score:', score, 'bounding_box:', bounding_box)
                        xmin, ymin, xmax, ymax = [int(x) for x in bounding_box]
                        if class_ID == 2 or scores[0][i] > max_thres:
                            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=1)
                            cv2.putText(frame, str(classes[class_ID])+':'+str(round(score, 2)), (xmin+10, ymin+10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
                        
                        if class_ID == 2:
                            hand = True
                        else:
                            if scores[0][i] > max_thres:
                                item_cnt[class_ID] += 1
                        
                for i in range(len(item_cnt)):
                    item_cnt[i] = min(item_cnt[i], max_item_cnt[i])
                        
                if not hand:
                    no_hand_cnt += 1
                else:
                    hand_cnt += 1
                    
                if hand_cnt >= 3:
                    start_trans = True
                    in_trans = True
                    hand_cnt = 0
                    no_hand_cnt = 0
                elif no_hand_cnt >= 3:
                    start_trans = False
                    hand_cnt = 0
                    no_hand_cnt = 0
                    
                if start_trans:
                    msg = 'Start Transaction'
                else:
                    msg = 'End Transaction'
                    if in_trans:
                        for i in range(len(item_cnt)):
                            change_item = item_cnt[i]-curr_item_cnt[i]
                            if change_item != 0:
                                char = ''
                                if change_item > 0:
                                    char = '+'
                                msg += ' '+classes[i]+': '+char+str(change_item)
                        in_trans = False
                        curr_item_cnt = item_cnt
                    else:
                        msg = pre_msg
                    
                    if not hand:
                        for i in range(len(item_cnt)):
                            curr_item_cnt[i] = max(curr_item_cnt[i], item_cnt[i])

                msg2 = ''
                for i in range(len(curr_item_cnt)):
                    msg2 += classes[i]+': '+str(curr_item_cnt[i])+' '
                
                cv2.putText(frame, msg, (40, 40), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, thickness=2, color=(255, 255, 255))
                cv2.putText(frame, msg2, (40, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
    return frame, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, msg, msg2

In [ ]:
"""
need add rtmp support for sagemaker, please follow this page
https://answers.opencv.org/question/180776/build-opencv-with-ffmpeg-support/
"""
url="rtmp://52.80.130.52/live/123456"

vcap = cv2.VideoCapture(url)
player = VideoPlayer()

hand_cnt = 0
no_hand_cnt = 0
start_trans = False
in_trans = False
curr_item_cnt = [0, 0]
max_item_cnt = [2, 2]
pre_msg = ''
pre_msg2 = ''
run_mode = 2  # 1 for original image, 2 for object detection
fps = 12  # 24

print("Video start")
while(True):
    # Capture frame-by-frame
    read_start = time.time()
    if run_mode == 1:
        for i in range(fps):
            ret, frame = vcap.read()
    if run_mode == 2:
        ret, frame = vcap.read()
#         ret, frame = vcap.read()
#         ret, frame = vcap.read()
    read_end = time.time()
    print('read time:', read_end-read_start)

    if frame is not None:
        start = time.time()
        frame = cv2.flip(frame, -1)
        
        # save image
        if run_mode == 1:
            filename = '../images/frame_'+str(time.time())+'.jpg'
            cv2.imwrite(filename, frame)

            print(filename)

        if run_mode == 2:
            # use SageMaker
            #with open(filename, 'rb') as image:
            #    f = image.read()
            #    b = bytearray(f)
            #result = sagemaker_endpoint.predict(b)
            #print(json.loads(result))

            # use GluonCV
            detect_start = time.time()
            class_IDs, scores, bounding_boxes = objectDetection.detect_image(frame)
#             class_IDs, scores, bounding_boxes = objectDetection.detect_image_yolo(frame)
            #print('class_IDs:', class_IDs)
            #print('scores:', scores)
            #print('bounding_boxes:', bounding_boxes)
            detect_end = time.time()
            print('detect time:', detect_end-detect_start)

            frame, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, msg, msg2 = detection_result_process(frame, objectDetection.classes, class_IDs, scores, bounding_boxes, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, max_item_cnt, pre_msg, pre_msg2)
            #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg:', msg)
            #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg2:', msg2)
            pre_msg = msg
            pre_msg2 = msg2

            # Display the resulting frame
            #cv2.imwrite(filename, frame)
            player(frame)
            end = time.time()
            #print('all time:', end-start)

            #t = 1
            #time.sleep(t)
            #vcap.set(cv2.CAP_PROP_POS_FRAMES, t*25)

            #t = 0.2
            #sleep_time = t-(end-start)
            #if sleep_time >= 0:
            #    time.sleep(sleep_time)
            #    vcap.set(cv2.CAP_PROP_POS_FRAMES, sleep_time*25)

            #skip_frame = max(0, 25-int(1/(end-start)))
            #print('skip_frame:', skip_frame)
            #if skip_frame > 0:
            #    vcap.set(cv2.CAP_PROP_POS_FRAMES, skip_frame)

        if run_mode == 1:
            t = 1
            time.sleep(t)
            
        #vcap.set(cv2.CAP_PROP_POS_FRAMES, t*25)
        #break

        # Press q to close the video windows before it ends if you want
        #if cv2.waitKey(22) & 0xFF == ord('q'):
        #    break
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")

Video start
read time: 0.006744861602783203
detect time: 0.09442663192749023


read time: 0.002573251724243164
detect time: 0.09891581535339355
read time: 0.0023696422576904297
detect time: 0.09820890426635742
read time: 0.004952192306518555
detect time: 0.10073661804199219
read time: 0.0008485317230224609
detect time: 0.09442377090454102
read time: 0.0009944438934326172
detect time: 0.08820843696594238
read time: 0.002886056900024414
detect time: 0.07908916473388672
read time: 0.0009808540344238281
detect time: 0.07656741142272949
read time: 0.0017771720886230469
detect time: 0.07590985298156738
read time: 0.0008411407470703125
detect time: 0.07764577865600586
read time: 0.0008363723754882812
detect time: 0.07209229469299316
read time: 0.0022449493408203125
detect time: 0.07383513450622559
read time: 0.0008909702301025391
detect time: 0.0737161636352539
read time: 0.0009756088256835938
detect time: 0.07236409187316895
read time: 0.0008745193481445312
detect time: 0.07555365562438965
read time: 0.00087738037109375
detect time: 0.07703185081481934
read time: 0.001

detect time: 0.07214570045471191
read time: 0.0014989376068115234
detect time: 0.07399702072143555
read time: 0.003465414047241211
detect time: 0.07215547561645508
read time: 0.0009646415710449219
detect time: 0.07562494277954102
read time: 0.0010747909545898438
detect time: 0.07164311408996582
read time: 0.000988006591796875
detect time: 0.07138991355895996
read time: 0.0012750625610351562
detect time: 0.07534980773925781
read time: 0.0034720897674560547
detect time: 0.07367372512817383
read time: 0.001016378402709961
detect time: 0.07484102249145508
read time: 0.000904083251953125
detect time: 0.07403039932250977
read time: 0.003871917724609375
detect time: 0.07437300682067871
read time: 0.0010237693786621094
detect time: 0.0763864517211914
read time: 0.001271963119506836
detect time: 0.0737156867980957
read time: 0.0022308826446533203
detect time: 0.07212090492248535
read time: 0.0009932518005371094
detect time: 0.07416558265686035
read time: 0.0012903213500976562
detect time: 0.072

detect time: 0.07455968856811523
read time: 0.0009636878967285156
detect time: 0.06981348991394043
read time: 0.0009415149688720703
detect time: 0.0765526294708252
read time: 0.004687786102294922
detect time: 0.07259964942932129
read time: 0.0011909008026123047
detect time: 0.07287049293518066
read time: 0.002321481704711914
detect time: 0.07394075393676758
read time: 0.0012257099151611328
detect time: 0.07499551773071289
read time: 0.002375364303588867
detect time: 0.07295060157775879
read time: 0.0011074542999267578
detect time: 0.07552766799926758
read time: 0.0010654926300048828
detect time: 0.07174348831176758
read time: 0.001283407211303711
detect time: 0.0724034309387207
read time: 0.00493621826171875
detect time: 0.0719902515411377
read time: 0.0010709762573242188
detect time: 0.07463765144348145
read time: 0.003396272659301758
detect time: 0.0737156867980957
read time: 0.0012972354888916016
detect time: 0.07764244079589844
read time: 0.004870891571044922
detect time: 0.0741195

detect time: 0.0754091739654541
read time: 0.0012514591217041016
detect time: 0.0735318660736084
read time: 0.0009672641754150391
detect time: 0.07117009162902832
read time: 0.0009877681732177734
detect time: 0.07582211494445801
read time: 0.0009069442749023438
detect time: 0.07122683525085449
read time: 0.0010755062103271484
detect time: 0.0743398666381836
read time: 0.004038095474243164
detect time: 0.0748131275177002
read time: 0.0009326934814453125
detect time: 0.07322931289672852
read time: 0.0011165142059326172
detect time: 0.07358646392822266
read time: 0.003016233444213867
detect time: 0.0726315975189209
read time: 0.0012464523315429688
detect time: 0.07032418251037598
read time: 0.0042498111724853516
detect time: 0.07345390319824219
read time: 0.0030891895294189453
detect time: 0.0716242790222168
read time: 0.003042936325073242
detect time: 0.0729529857635498
read time: 0.0012485980987548828
detect time: 0.0735788345336914
read time: 0.005340099334716797
detect time: 0.0719683

detect time: 0.07357001304626465
read time: 0.003937244415283203
detect time: 0.07122397422790527
read time: 0.0012655258178710938
detect time: 0.07828307151794434
read time: 0.0012803077697753906
detect time: 0.07503557205200195
read time: 0.0017995834350585938
detect time: 0.07630729675292969
read time: 0.0009469985961914062
detect time: 0.0702371597290039
read time: 0.0012216567993164062
detect time: 0.07437825202941895
read time: 0.0037860870361328125
detect time: 0.07096433639526367
read time: 0.0009655952453613281
detect time: 0.07189035415649414
read time: 0.0013093948364257812
detect time: 0.0744619369506836
read time: 0.0009746551513671875
detect time: 0.07375836372375488
read time: 0.0009729862213134766
detect time: 0.07102704048156738
read time: 0.0009758472442626953
detect time: 0.07348895072937012
read time: 0.003504514694213867
detect time: 0.07351946830749512
read time: 0.0033812522888183594
detect time: 0.07094597816467285
read time: 0.004038095474243164
detect time: 0.